In [1]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [2]:
pip install --upgrade pydantic


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
from langchain.document_loaders import PyPDFLoader
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate


In [4]:
!pip install PyPDF2 pdfplumber

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [27]:
# Load PDF
loader_1 = PyPDFLoader("10-Q4-2024-As-Filed.pdf")
document_1 = loader_1.load()
loader_2 = PyPDFLoader("10-k-2022.pdf")
document_2 = loader_2.load()
loader_3 = PyPDFLoader("10k-2020.pdf")
document_3 = loader_3.load()
loader_4 = PyPDFLoader("10k-2018.pdf")
document_4 = loader_4.load()
loader_5 = PyPDFLoader("10k-2016.pdf")
document_5 = loader_5.load()

In [28]:
document_1

[Document(metadata={'source': '10-Q4-2024-As-Filed.pdf', 'page': 0}, page_content='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☒    ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended September\xa028, 2024\nor\n☐    TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  to \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-2404110\n(State or other jurisdiction\nof incorporation or organization)(I.R.S. Employer Identification No.)\nOne Apple Park Way\nCupertino , California 95014\n(Address of principal executive offices) (Zip Code)\n(408) 996-1010\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12

In [29]:
document_2

[Document(metadata={'source': '10-k-2022.pdf', 'page': 0}, page_content='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☒ ANNUAL  REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended September 24, 2022\nor\n☐ TRANSITION REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from              to             .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-24041 10\n(State or other jurisdiction\nof incorporation or organization)(I.R.S. Employer Identification No.)\nOne Apple Park W ay\nCupertino, California 95014\n(Address of principal executive offices) (Zip Code)\n(408) 996-1010\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class Trading symbol(s) Name of each exchange on which reg

In [34]:
def page_finder(document):
    start_page = None
    end_page = None
    # Find the page number where content starts with "Item 8"
    for doc in document:
        if doc.page_content.startswith("Item 8"):
            start_page = doc.metadata['page']
        break

    # Find the page number where content starts with "Item 8"
    for doc in document:
        if doc.page_content.startswith("Item 9"):
            end_page= doc.metadata['page']
        break
    return start_page,end_page

In [35]:
# # Find the page number where content starts with "Item 8"
# for doc in documents:
#     if doc.page_content.startswith("Item 8"):
#         print(f"Item 8 starts on page {doc.metadata['page']}")
#         start_page = doc.metadata['page']
#         break

In [36]:
# for doc in documents:
#     if doc.page_content.startswith("Item 9"):
#         print(f"Item 9 starts on page {doc.metadata['page']}")
#         end_page= doc.metadata['page']
#         break

In [37]:
def page_executor(document):
    start_page,end_page = page_finder(document)
    if start_page is not None and end_page is not None:
        extracted_documents = []
        for doc in document:
            if start_page <= doc.metadata['page'] < end_page:
                extracted_documents.append(doc)
    return extracted_documents

In [38]:
# if start_page is not None and end_page is not None:
#     extracted_documents = []
#     for doc in documents:
#         if start_page <= doc.metadata['page'] < end_page:
#             extracted_documents.append(doc)

In [42]:
def page_finder(document):
    # Initialize variables to store page numbers
    start_page = None
    end_page = None

    # Find the page number where content starts with "Item 8"
    for doc in document:
        if doc.page_content.startswith("Item 8"):
            start_page = doc.metadata['page']
            break  # Exit the loop after finding the first match

    # Find the page number where content starts with "Item 9"
    for doc in document:
        if doc.page_content.startswith("Item 9"):
            end_page = doc.metadata['page']
            break  # Exit the loop after finding the first match

    # If either page is still None (should not happen if items are guaranteed)
    if start_page is None or end_page is None:
        raise ValueError("Could not find 'Item 8' or 'Item 9' in the document")

    return start_page, end_page


def page_executor(document):
    # Find the start and end pages
    start_page, end_page = page_finder(document)

    # Extract documents between the start and end pages
    extracted_documents = [
        doc for doc in document
        if start_page <= doc.metadata['page'] < end_page
    ]

    return extracted_documents

In [43]:
extract_2024 = page_executor(document_1)

In [44]:
extract_2024

[Document(metadata={'source': '10-Q4-2024-As-Filed.pdf', 'page': 30}, page_content='Item 8. Financial Statements and Supplementary Data\nIndex to Consolidated Financial Statements Page\nConsolidated Statements of Operations for the years ended September 28, 2024, September 30, 2023 and \nSeptember 24, 2022 29\nConsolidated Statements of Comprehensive Income for the years ended September 28, 2024, September 30, \n2023 and September 24, 2022 30\nConsolidated Balance Sheets as of September 28, 2024 and September 30, 2023 31\nConsolidated Statements of Shareholders’ Equity for the years ended September 28, 2024, September 30, 2023 \nand September 24, 2022 32\nConsolidated Statements of Cash Flows for the years ended September 28, 2024, September 30, 2023 and \nSeptember 24, 2022 33\nNotes to Consolidated Financial Statements 34\nReports of Independent Registered Public Accounting Firm 48\nAll financial statement schedules have been omitted, since the required information is not applicable 

In [45]:
extract_2022 = page_executor(document_2)

In [46]:
extract_2018 = page_executor(document_3)
extract_2016 = page_executor(document_4)
extract_2014 = page_executor(document_5)

In [47]:
extract_2014

[Document(metadata={'source': '10k-2016.pdf', 'page': 40}, page_content='Item 8.Financial Statement s and Supplementary DataIndex to Consolidated Financial Statements\n\xa0 Page Consolidated Statements of Operation\ns for the years ended September 24, 2016, September 26, 2015 and   September 27, 2014\xa0 39 Consolidated Statements of Comp\nrehensive Income for the years ended September 24, 2016, September 26, 2015   and September 27, 2014\xa0 40 Consolidated Balance Sheets a\ns of September 24, 2016 and September 26, 2015\xa0 41 Consolidated Statements of Shareho\nlders’ Equity for the years ended September 24, 2016, September 26, 2015 and September 27, 2014\xa0 42 Consolidated Statements of Cash F\nlows for the years ended September 24, 2016, September 26, 2015 and   September 27, 2014\xa0 43 Notes to Consolidated Fin\nancial Statements\xa0 44 Selected Quarterly Financial \nInformation (Unaudited)\xa0 69 Reports of Ernst & Young LLP, Independent Reg\nistered Public Accounting Firm\xa0

In [11]:
!pip install langchain-openai

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
# from langchain.chains import SequentialChain
# from langchain.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI
# from langchain.chains import LLMChain
# from langchain.agents import AgentExecutor, Tool
# from langchain.agents import ZeroShotAgent

# # Define the system prompt
# extract_prompt = PromptTemplate(
#     input_variables=["document"],
#     template="""
#     The following is a section of a 10-K filing:
#     {document}

#     Extract the balance sheet, cashflow and shareholder's equity as a structured table. Include column names and all rows. Prepare different tables for balance sheet, cashflow and shareholder's equity
#     Provide the table as a JSON object.
#     """
# )

# llm = ChatOpenAI(model="gpt-4o-mini",temperature=0.5, openai_api_key=key_string)

# # Define the first chain
# extract_chain = LLMChain(
#     llm=llm,
#     prompt=extract_prompt,
#     output_key="balance_sheet",  # This key will pass to the next chain
# )

# # Define the second prompt
# convert_prompt = PromptTemplate(
#     input_variables=["balance_sheet"],
#     template="""
#     Convert the following balance sheet, cashflow and shareholder's equity JSON into a TSV format. Return only the TSV data. Separate the 3 tables.

#     JSON:
#     {balance_sheet}
#     """
# )

# # Define the second chain
# convert_chain = LLMChain(
#     llm=llm,
#     prompt=convert_prompt,
#     output_key="csv_output",
# )

# # Combine the chains
# sequential_chain = SequentialChain(
#     chains=[extract_chain, convert_chain],
#     input_variables=["document"],
#     output_variables=["csv_output"],
# )

# # Define tools for the agent
# tools = [
#     Tool(
#         name="Extract and Convert Financial Data",
#         func=sequential_chain.run,
#         description="Extracts financial data from a 10-K filing and converts it to TSV format."
#     )
# ]
# # Create the agent
# agent = ZeroShotAgent(
#     llm=llm,
#     tools=tools,
#     verbose=True
# )

# # Create the agent executor
# agent_executor = AgentExecutor(
#     tools=tools,
#     agent=agent,
#     verbose=True
# )

# # Now you can use the agent_executor to process documents
# document = "Your 10-K filing section here"
# result = agent_executor({"document": document})
# print(result["csv_output"])

/var/folders/09/kkd4kch12b35yvx1xk3m4gr00000gn/T/ipykernel_62091/54726663.py:63: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~create_react_agent` instead.
  agent = ZeroShotAgent(


ValidationError: 1 validation error for ZeroShotAgent
llm_chain
  Field required [type=missing, input_value={'llm': ChatOpenAI(client...'])>)], 'verbose': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [122]:
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.agents import initialize_agent

# Define the system prompt
extract_prompt = PromptTemplate(
    input_variables=["document"],
    template="""
    The following is a section of a 10-K filing:
    {document}

    Extract the consolidated balance sheetas a structured table. Include column names and all rows. 
    Provide the table as a JSON object.
    """
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5, openai_api_key=key_string)

# Define the first chain
extract_chain = LLMChain(
    llm=llm,
    prompt=extract_prompt,
    output_key="balance_sheet",  # This key will pass to the next chain
)

# Define the second prompt
convert_prompt = PromptTemplate(
    input_variables=["balance_sheet"],
    template="""
    Convert the following balance sheet JSON into a CSV format. Return only the TSV data. make downlodable TSV file

    JSON:
    {balance_sheet}
    """
)

# Define the second chain
convert_chain = LLMChain(
    llm=llm,
    prompt=convert_prompt,
    output_key="csv_output",
)

# Combine the chains
sequential_chain = SequentialChain(
    chains=[extract_chain, convert_chain],
    input_variables=["document"],
    output_variables=["csv_output"],
)

# Create an LLMChain for the agent
agent_llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["input"],
        template="Process the following input: {input}"
    )
)

# Define tools for the agent
tools = [
    Tool(
        name="Extract and Convert Financial Data",
        func=sequential_chain.run,
        description="Extracts financial data from a 10-K filing and converts it to TSV format."
    )
]

# Create the agent
agent = initialize_agent(
    llm = llm,
    #llm_chain=agent_llm_chain,
    tools=tools,
    verbose=True
)

# # Create the agent executor
# agent_executor = AgentExecutor(
#     agent=agent,
#     tools=tools,
#     handle_parsing_errors=True,
#     verbose=True
# )

# Now you can use the agent_executor to process documents
# document = "Your 10-K filing section here"
# result = agent_executor({"input": document})
# print(result["csv_output"])

In [124]:
agent.run("give me insights")



> Entering new AgentExecutor chain...
To provide insights, I need to extract relevant financial data from a 10-K filing. I'll start by determining which specific data points or sections might be useful for generating insights. Common areas of interest include financial performance metrics, risk factors, and management discussion. 

Action: Extract and Convert Financial Data  
Action Input: "10-K filing"  # Assuming we need to specify which company's filing or year; I'll need to clarify that. 

Observation: Below is the TSV representation of the provided JSON structure for a consolidated balance sheet. You can download the TSV file using the link provided.

```plaintext
Category	Subcategory	Item	Value
Assets	CurrentAssets	CashAndCashEquivalents	0
Assets	CurrentAssets	AccountsReceivable	0
Assets	CurrentAssets	Inventory	0
Assets	CurrentAssets	PrepaidExpenses	0
Assets	CurrentAssets	OtherCurrentAssets	0
Assets	CurrentAssets	TotalCurrentAssets	0
Assets	NonCurrentAssets	PropertyPlantAndEqui

KeyboardInterrupt: 

In [ ]:
result = agent_executor({"input":extract_2014})

SyntaxError: ':' expected after dictionary key (3708943571.py, line 1)

In [95]:
result

"```\n# Balance Sheet\nitem\tcategory\tsubcategory\t2024\t2023\t2022\t2020\t2019\t2018\t2017\t2016\t2015\t2014\nAssets\tCurrent Assets\tCash and Cash Equivalents\t29943\t29965\t23646\t38016\t48844\t20989\t20484\t21120\t20120\t\nAssets\tCurrent Assets\tMarketable Securities\t35228\t31590\t24658\t52927\t51713\t53892\t46871\t46671\t20481\t\nAssets\tCurrent Assets\tAccounts Receivable, Net\t33410\t29508\t28184\t16120\t22926\t23886\t17874\t16849\t16849\t\nAssets\tCurrent Assets\tInventories\t7286\t6331\t4946\t4061\t4106\t4855\t4855\t2349\t2349\t\nAssets\tTotal Assets\t\t364980\t352583\t352755\t323888\t338516\t365725\t375319\t321686\t290345\t\nLiabilities\tCurrent Liabilities\tAccounts Payable\t68960\t62611\t64115\t42296\t46236\t44242\t44242\t35490\t\t\t\nLiabilities\tTotal Liabilities\t\t308030\t290437\t302083\t258549\t248028\t258578\t241272\t193437\t\t\t\n\n# Cash Flow\nitem\tcategory\tsubcategory\t2024\t2023\t2022\t2020\t2019\t2018\t2017\t2016\t2015\t2014\nCash Flows from Operating Activi

In [96]:
with open('balance_sheet_17.tsv', 'w') as file:
    file.write(result)

print("The balance sheet data has been saved to 'balance_sheet_3.tsv'.")

The balance sheet data has been saved to 'balance_sheet_3.tsv'.
